In [ ]:
!pip install tkcalendar
!pip install emoji

In [6]:
!pip install tkcalendar
!pip install emoji
!pip install matplotlib
!pip install pandas
from tkinter import *
from tkinter import ttk
from tkinter import messagebox
from tkinter.ttk import Notebook
from tkcalendar import DateEntry
from tkinter.tix import *
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import pandas as pd
import datetime as dt
import calendar
import numpy as np
from emoji import emojize

    





def Addexpense ():
    ##create list for df
    column_a_list = []
    column_b_list = []
    column_c_list = []
    column_d_list = []
    column_e_list = []
    column_f_list = []

    ## get values
    a = EDate.get()
    b = Title.get()
    c = Type.get()
    d = '{:,}'.format(float(Expense.get()))
    e = '{:,}'.format(float(Income.get()))
    f = '{:,}'.format((float(Income.get()) - float(Expense.get())))

    data = [a,b,c,d,e,f]
 

    TVExpense.insert('', 'end', values=data)
    ## reset value to 0 to prevent error 
    Expense.set(0.0)
    Income.set(0.0)
    ## create dataframe for plot
    for child in TVExpense.get_children():
        column_a_list.append(TVExpense.item(child)["values"][0])            
        column_b_list.append(TVExpense.item(child)["values"][1])  
        column_c_list.append(TVExpense.item(child)["values"][2])  
        column_d_list.append(TVExpense.item(child)["values"][5])
        column_e_list.append(TVExpense.item(child)["values"][3])
        column_f_list.append(TVExpense.item(child)["values"][4])
    full_treeview_data_dict = {'Dates': column_a_list, 'Transaction': column_b_list, 'Category': column_c_list, 'Balance': column_d_list, 'Expense': column_e_list, 'Income': column_f_list}
    treeview_df = pd.DataFrame.from_dict(full_treeview_data_dict)

    treeview_df['Dates'] = pd.to_datetime(treeview_df['Dates']).dt.date
    treeview_df['Balance'] = [float(str(i).replace(",","")) for i in treeview_df['Balance']]
    treeview_df['Income'] = [float(str(i).replace(",","")) for i in treeview_df['Income']]
    treeview_df['Expense'] = [float(str(i).replace(",","")) for i in treeview_df['Expense']]

    figure1 = plt.Figure(figsize=(15.3, 5.5), dpi=80)
    ax1 = figure1.add_subplot(111)
    bar1 = FigureCanvasTkAgg(figure1,bottom_frame)
    bar1.get_tk_widget().grid(row=0, column=0, padx=1, pady=0)
    treeview_df1 = treeview_df[['Dates', 'Balance']].groupby('Dates').sum()
    treeview_df1 .plot(kind='bar', color='aliceblue', legend=True, ax=ax1)
    ax1.set_title('Daily Balance')
    ax1.set_facecolor('skyblue')
    figure1.tight_layout()

   
    ### monthly tab
    TVList_2 = ['Year', 'Month', 'Expense', 'Income', 'Balance']
    TVExpense_2 = ttk.Treeview(right_frame_2, column=TVList_2, show='headings', height=10)
    for i in TVList_2:
        TVExpense_2.heading(i, text= i.title())
    TVExpense_2.grid(row=1, column=2, padx = 10, pady=5, sticky ='w', columnspan = 5)
    
    ### yearly tab
    TVList_3 = ['Year', 'Expense', 'Income', 'Balance']
    TVExpense_3 = ttk.Treeview(right_frame_3, column=TVList_3, show='headings', height=10)
    for i in TVList_3:
        TVExpense_3.heading(i, text= i.title())
    TVExpense_3.grid(row=1, column=2, padx = 10, pady=5, sticky ='w', columnspan = 4)


    

        ######### create monthly dataframe ###########
    def Addmonthly():
        treeview_df2 = treeview_df[['Dates', 'Income', 'Expense', 'Balance']].groupby(['Dates'], as_index=False).agg({'Expense':'sum', 'Income':'sum', 'Balance':'sum'})
        treeview_df2['Month #'] = pd.DatetimeIndex(treeview_df2['Dates']).month
        treeview_df2['Month'] = treeview_df2['Month #'].apply(lambda x: calendar.month_abbr[x])
        treeview_df2['Year'] = pd.DatetimeIndex(treeview_df2['Dates']).year
        treeview_df2_use = treeview_df2[['Year','Month #','Month', 'Income', 'Expense', 'Balance']].groupby(['Year','Month #','Month'], as_index=False).agg({'Expense':'sum', 'Income':'sum', 'Balance':'sum'})
        treeview_df2_use.sort_values(by=['Year','Month #'], ascending=False)
        for _ in range(len(treeview_df2_use.index.values)): # use for loop to get values in each line, _ is the number of line.
            TVExpense_2.insert('','end',value=tuple(treeview_df2_use.iloc[_,[0,2,3,4,5]].values)) # [_,[1,2]] represents that you will get the values of second column and third column for each line.
        
        figure2 = plt.Figure(figsize=(14, 6), dpi=80)
        ax2 = figure2.add_subplot(111)
        bar2 = FigureCanvasTkAgg(figure2,bottom_frame_2)
        bar2.get_tk_widget().grid(row=0, column=0, padx=1, pady=0)
        df =  treeview_df2_use[['Year', 'Month #', 'Month', 'Balance']].groupby([ 'Year', 'Month #', 'Month']).agg({'Balance':'sum'})
        df .plot(kind='bar',color='aliceblue', legend=True, ax=ax2)
        ax2.set_title('Monthly Balance')
        ax2.set_facecolor('skyblue')
        figure2.tight_layout()


        h = df["Balance"].iloc[-1]
        Save.set(h)
        ESave = ttk.Entry (left_frame_2, textvariable=Save, font=('calibri',15),background='lightskyblue1') 
        ESave.grid(row=1, column=1, padx=4, pady=5,sticky='w')

        
        def Difference():
            j = h - Target.get() 
            Diff.set(j)
            EDiff = ttk.Entry (left_frame_2, textvariable=Save, font=('calibri',15),background='lightskyblue1') 
            EDiff.grid(row=1, column=1, padx=4, pady=5,sticky='w')
            if j > 0: 
                Result.set(emojize(":smiling_face_with_sunglasses:") + "Great Job!" + emojize(":smiling_face_with_sunglasses:"))
                EResult = ttk.Entry (left_frame_2, textvariable=Result, font=('calibri',15),background='lightskyblue1')
                EResult.grid(row=4, column=1, padx=4, pady=5,sticky='w')
            if j < 0: 
                Result.set(emojize(":slightly_frowning_face:") + "You can do better!" + emojize(":slightly_frowning_face:"))
                EResult = ttk.Entry (left_frame_2, textvariable=Result, font=('calibri',15),background='lightskyblue1')
                EResult.grid(row=4, column=1, padx=4, pady=5,sticky='w')            
            if j == 0: 
                Result.set(emojize(":grinning_face_with_smiling_eyes:") + "It's Okay" + emojize(":grinning_face_with_smiling_eyes:"))
                EResult = ttk.Entry (left_frame_2, textvariable=Result, font=('calibri',15),background='lightskyblue1')
                EResult.grid(row=4, column=1, padx=4, pady=5,sticky='w')                        
            
            
        BF4Add = ttk.Button (left_frame_2, text = 'Calculate', command = Difference)
        BF4Add.grid(row=3, column=1, padx = 10, pady=5, sticky ='e', ipadx=10, ipady=5)
        
  
    
    def Addyearly():
        treeview_df3 = treeview_df[['Dates', 'Income', 'Expense', 'Balance']].groupby(['Dates'], as_index=False).agg({'Expense':'sum', 'Income':'sum', 'Balance':'sum'})
        treeview_df3['Month'] = pd.DatetimeIndex(treeview_df3['Dates']).month
        treeview_df3['Month'] = treeview_df3['Month'].apply(lambda x: calendar.month_abbr[x])
        treeview_df3['Year'] = pd.DatetimeIndex(treeview_df3['Dates']).year
        treeview_df3_use = treeview_df3[['Year', 'Income', 'Expense', 'Balance']].groupby(['Year'], as_index=False).agg({'Expense':'sum', 'Income':'sum', 'Balance':'sum'})
        treeview_df3_use.sort_values(by=['Year'], ascending=False)
        for _ in range(len(treeview_df3_use.index.values)): # use for loop to get values in each line, _ is the number of line.
            TVExpense_3.insert('','end',value=tuple(treeview_df3_use.iloc[_,[0,1,2,3]].values)) # [_,[1,2]] represents that you will get the values of second column and third column for each line.
        
        figure3 = plt.Figure(figsize=(11.5, 6), dpi=80)
        ax3 = figure3.add_subplot(111)
        bar3 = FigureCanvasTkAgg(figure3,bottom_frame_3)
        bar3.get_tk_widget().grid(row=0, column=0, padx=1, pady=0)
        df3 =  treeview_df3_use[['Year', 'Balance']].groupby([ 'Year']).agg({'Balance':'sum'})
        df3 .plot(kind='bar', color='aliceblue', legend=True, ax=ax3)
        ax3.set_title('Yearly Balance')
        ax3.set_facecolor('skyblue')
        figure3.tight_layout()
    
    
    BF2Add = ttk.Button (right_frame_2, text = 'Add', command = Addmonthly)
    BF2Add.grid(row=1, column=0, padx = 4, pady=5, sticky ='nw', ipadx=5, ipady=5)
    
    BF3Add = ttk.Button (right_frame_3, text = 'Add', command = Addyearly)
    BF3Add.grid(row=1, column=0, padx = 4, pady=5, sticky ='nw', ipadx=5, ipady=5)
        

    #### saving show#####
    ###need to create entry box and variable to capture below before writing in function to remove insert error
    g = treeview_df1['Balance'].sum()
    Saving.set(g)
    ESaving = ttk.Entry (tool_bar, textvariable=Saving, font=('calibri',10),background='lightskyblue1') 
    ESaving.grid(row=6, column=1, padx=4, pady=5,sticky='w')

    


    
GUI = Tk()
GUI.title('Expense and Income Tracker')
GUI.config(bg="skyblue")
GUI.geometry('950x600')



# main
Tab = Notebook(GUI)

F1 = Frame(Tab, width = 950, height = 700)
F2 = Frame(Tab, width = 950, height = 700)
F3 = Frame(Tab, width = 950, height = 700)

Tab.add(F1, text='Daily')
Tab.add(F2, text='Monthly')
Tab.add(F3, text='Yearly')

Tab.pack(fill=BOTH, expand=1)

#-------------------------------Tab1 daily------------------------------------------------
main_frame = ttk.Frame(F1)
main_frame.pack(fill=BOTH, expand=1)

# canvas
my_canvas = Canvas(main_frame)
my_canvas.pack(side=LEFT, fill=BOTH, expand=1)

# scrollbar
my_scrollbar = ttk.Scrollbar(main_frame, orient=VERTICAL, command=my_canvas.yview)
my_scrollbar.pack(side=RIGHT, fill=Y)

# configure the canvas
my_canvas.configure(yscrollcommand=my_scrollbar.set)
my_canvas.bind('<Configure>', lambda e: my_canvas.configure(scrollregion=(0,0,950,800)))



#### split frame
my_canvas.config(bg="skyblue")

left_frame = Frame(my_canvas, width = 90, height = 100, bg='lightskyblue1')
left_frame.grid(row=0, column=0, padx=3, pady=5)

right_frame = Frame(my_canvas, width= 90, height = 100, bg='lightskyblue1')
right_frame.grid(row=0, column=1, padx=3, pady=10)

bottom_frame = Frame(my_canvas, width= 800, height = 400, bg='lightskyblue1')
bottom_frame.grid(row=1, column=1, padx=5, pady=5)

tool_bar = Frame(left_frame, width = 100, height =300, bg="lightskyblue1")
tool_bar.grid(row=1, column=0, padx=5, pady=5)
GUI.state('zoomed')



#labels

#create frames and label in left frame
Label(left_frame, text="Expenses", font=('calibri',12),background='lightblue', foreground='black').grid(row=0, column=0, padx=5, pady=5)
Label(right_frame, text="Summary", font=('calibri',12),background='lightblue', foreground='black').grid(row=0, column=3, padx=4, pady=10, sticky='n')
                                                                               
                                                                                                        
                                                                                                        

#---------------1-----------------------
LDate = ttk.Label (tool_bar, text='Date', font=('calibri',10), background='lightskyblue1')
LDate.grid(row=0, column=0, padx=4, pady=5, sticky='w')

EDate = DateEntry(tool_bar, width=18, background = 'skyblue', foreground = 'black',font=('calibri',10))
EDate.grid(row=0, column=1, padx=4, pady=5, sticky='w')

#---------------2-----------------------
LTitle = ttk.Label (tool_bar, text='Transaction', font=('calibri',10),background='lightskyblue1')
LTitle.grid(row=1, column=0, padx=4, pady=5, sticky='w')

Title = StringVar()
ETitle = ttk.Entry (tool_bar, textvariable=Title, font=('calibri',10),background='lightskyblue1')
ETitle.grid(row=1, column=1, padx=4, pady=5, sticky='w')

#---------------3-----------------------


LType = ttk.Label (tool_bar, text='Category', font=('calibri',10),background='lightskyblue1')
LType.grid(row=2, column=0, padx=4, pady=5, sticky='w')


OPTIONS = ["Salary", "Food", "Transportation",
           "Entertainment", "Sport", "Necessities", "Shopping", "Others"]

Type = StringVar(tool_bar)

Type.set("Choose") # default value

# EType = OptionMenu(tool_bar, Type, *OPTIONS)
EType = ttk.Combobox(tool_bar, width = 18, textvariable=Type, values=OPTIONS, font=('calibri',10),background='lightskyblue1')
EType.grid(row=2, column=1, padx=4, pady=5, sticky='w')




#---------------4-----------------------
LExpense = ttk.Label (tool_bar, text='Expense', font=('calibri',10),background='lightskyblue1')
LExpense.grid(row=3, column=0, padx=4, pady=5,sticky='w')

Expense = DoubleVar()
EExpense = ttk.Entry (tool_bar, textvariable=Expense, font=('calibri',10),background='lightskyblue1')
EExpense.grid(row=3, column=1, padx=4, pady=5,sticky='w')

#---------------5-----------------------
LIncome = ttk.Label (tool_bar, text='Income', font=('calibri',10),background='lightskyblue1')
LIncome.grid(row=4, column=0, padx=4, pady=5,sticky='w')

Income = DoubleVar()
EIncome = ttk.Entry (tool_bar, textvariable=Income, font=('calibri',10),background='lightskyblue1')
EIncome.grid(row=4, column=1, padx=4, pady=5,sticky='w')

#---------------6-----------------------


# ESavingAdd = ttk.Button (tool_bar, text = 'Saving', command = S)
# ESavingAdd.grid(row=7, column=1, padx=4, pady=5,sticky='w', ipadx=5, ipady=5)




# Create style Object
style = ttk.Style()

style.configure('TButton', font =
               ('calibri', 12, 'bold'),
                    borderwidth = '5')


# Changes will be reflected
# by the movement of mouse.
style.map('TButton', foreground = [('active', '!disabled', 'skyblue')],
                     background = [('active', 'black')])


BF1Add = ttk.Button (tool_bar, text = 'Add', command = Addexpense)
BF1Add.grid(row=5, column=1, padx = 4, pady=5, sticky ='w', ipadx=5, ipady=5)


LSaving = ttk.Label (tool_bar, text='Total Saving', font=('calibri',10),background='lightskyblue1')
LSaving.grid(row=6, column=0, padx=4, pady=5,sticky='w')


Saving = DoubleVar()
ESaving = ttk.Entry (tool_bar, textvariable=Saving, font=('calibri',10),background='lightskyblue1') 
ESaving.grid(row=6, column=1, padx=4, pady=5,sticky='w')



ttk.Style().configure("Treeview", background="lightskyblue1", 
foreground="black", fieldbackground="lightskyblue1")

TVList = ['Date', 'Transaction', 'Category', 'Expense', 'Income', 'Balance']
TVExpense = ttk.Treeview(right_frame, column=TVList, show='headings', height=10)
for i in TVList:
    TVExpense.heading(i, text= i.title())
TVExpense.grid(row=1, column=2, padx = 10, pady=5, sticky ='w', columnspan = 3)








#-------------------------------Tab2 monthly------------------------------------------------
main_frame_2 = ttk.Frame(F2)
main_frame_2.pack(fill=BOTH, expand=1)

# canvas
my_canvas_2 = Canvas(main_frame_2)
my_canvas_2.pack(side=LEFT, fill=BOTH, expand=1)

# scrollbar
my_scrollbar_2 = ttk.Scrollbar(main_frame_2, orient=VERTICAL, command=my_canvas_2.yview)
my_scrollbar_2.pack(side=RIGHT, fill=Y)

# configure the canvas
my_canvas_2.configure(yscrollcommand=my_scrollbar_2.set)
my_canvas_2.bind('<Configure>', lambda e: my_canvas_2.configure(scrollregion=(0,0,950,800)))


#### split frame
my_canvas_2.config(bg="skyblue")

######## second tab right frame ############
left_frame_2 = Frame(my_canvas_2, width = 150, height = 90, bg='lightskyblue1')
left_frame_2.grid(row=0, column=2, padx=3, pady=10, sticky='n')

LTarget = ttk.Label (left_frame_2, text='Target', font=('calibri',15),background='lightskyblue1')
LTarget.grid(row=0, column=0, padx=4, pady=5,sticky='w')

Target = DoubleVar()
ETarget = ttk.Entry (left_frame_2, textvariable=Target, font=('calibri',15),background='lightskyblue1')
ETarget.grid(row=0, column=1, padx=4, pady=5,sticky='w')


LSave = ttk.Label (left_frame_2, text='Save', font=('calibri',15),background='lightskyblue1')
LSave.grid(row=1, column=0, padx=4, pady=5,sticky='w')

Save = DoubleVar()
ESave = ttk.Entry (left_frame_2, textvariable=Save, font=('calibri',15),background='lightskyblue1')
ESave.grid(row=1, column=1, padx=4, pady=5,sticky='w')


LDiff = ttk.Label (left_frame_2, text='Differences', font=('calibri',15),background='lightskyblue1')
LDiff.grid(row=2, column=0, padx=4, pady=5,sticky='w')

Diff = DoubleVar()
EDiff = ttk.Entry (left_frame_2, textvariable=Diff, font=('calibri',15),background='lightskyblue1')
EDiff.grid(row=2, column=1, padx=4, pady=5,sticky='w')


LResult = ttk.Label (left_frame_2, text='Result', font=('calibri',15),background='lightskyblue1')
LResult.grid(row=4, column=0, padx=4, pady=5,sticky='w')

Result = StringVar()
EResult = ttk.Entry (left_frame_2, textvariable=Result, font=('calibri',15),background='lightskyblue1')
EResult.grid(row=4, column=1, padx=4, pady=5,sticky='w')







####### second tab left frame #############
right_frame_2 = Frame(my_canvas_2, width= 90, height = 90, bg='lightskyblue1')
right_frame_2.grid(row=0, column=1, padx=3, pady=10)

Label(right_frame_2, text="Summary", font=('calibri',12),background='lightblue', foreground='black').grid(row=0, column=4, padx=4, pady=0, sticky='n')


bottom_frame_2 = Frame(my_canvas_2, width= 400, height = 610, bg='lightskyblue1')
bottom_frame_2.grid(row=1, column=1, padx=3, pady=10)

# tool_bar_2 = Frame(left_frame_2, width = 100, height =300, bg="lightskyblue1")
# tool_bar_2.grid(row=1, column=0, padx=5, pady=5)





#-------------------------------Tab3 monthly------------------------------------------------
main_frame_3 = ttk.Frame(F3)
main_frame_3.pack(fill=BOTH, expand=1)

# canvas
my_canvas_3 = Canvas(main_frame_3)
my_canvas_3.pack(side=LEFT, fill=BOTH, expand=1)

# scrollbar
my_scrollbar_3 = ttk.Scrollbar(main_frame_3, orient=VERTICAL, command=my_canvas_3.yview)
my_scrollbar_3.pack(side=RIGHT, fill=Y)

# configure the canvas
my_canvas_3.configure(yscrollcommand=my_scrollbar_3.set)
my_canvas_3.bind('<Configure>', lambda e: my_canvas_3.configure(scrollregion=(0,0,950,800)))



#### split frame
my_canvas_3.config(bg="skyblue")

right_frame_3 = Frame(my_canvas_3, width= 100, height = 90, bg='lightskyblue1')
right_frame_3.grid(row=0, column=1, padx=3, pady=10)

Label(right_frame_3, text="Summary", font=('calibri',12),background='lightblue', foreground='black').grid(row=0, column=3, padx=375, pady=0, sticky='n')
                         

bottom_frame_3 = Frame(my_canvas_3, width= 400, height = 610, bg='lightskyblue1')
bottom_frame_3.grid(row=1, column=1, padx=3, pady=10)



GUI.mainloop()

